In [1]:
import pandas as pd
#from keras.preprocessing import image
from matplotlib import pyplot
from matplotlib.pyplot import imshow
import split_data as sd
import basic_model as n
import numpy as np
from sklearn.metrics import confusion_matrix
#import Network as net
import vectorised as v
from random import randint
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
config=tf.ConfigProto()
config.gpu_options.allow_growth=True

Using TensorFlow backend.


In [2]:
data=pd.read_csv('/home/snehao/dog_breed/Image_Classification/labels.csv')
print(data.head())
targets_series = pd.Series(data['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [3]:
x_train,x_test,y_train,y_test=sd.split_data(data,one_hot_labels,.20)
#y_test1=x_test['breed'].values
#print(x_train.nlargest(5,x_train.groupby(['breed']).size()))
x_val,x_test,y_val,y_test=sd.split_data(x_test,y_test,.50)
#print(x_train.groupby(['breed']).size().sort_values())
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(1022, 2)
(1022, 120)
(1023, 2)
(1023, 120)


In [4]:
x=[]
y=[]
#for i in range(100):
for i in range(x_train.shape[0]):
    img = image.load_img('/home/snehao/dog_breed/train/'+x_train.iloc[i][0]+'.jpg',target_size=(224,224,3))
    arr=image.img_to_array(img)
    x.append(arr/255)
    y.append(y_train[i])
x=np.array(x)
y=np.array(y)
print('x_shape',x.shape)
print('y_shape',y.shape)

x_shape (8177, 224, 224, 3)
y_shape (8177, 120)


In [5]:
x_val_images=[]
#for i in range(100):
for i in range(x_val.shape[0]): 
    img = image.load_img('/home/snehao/dog_breed/train/'+x_val.iloc[i][0]+'.jpg',target_size=(224,224,3))
    arr=image.img_to_array(img)
    x_val_images.append(arr/255)
    
x_val_images=np.array(x_val_images)
print('y_shape',y_val.shape)
print('x_shape',x_val_images.shape)


y_shape (1022, 120)
x_shape (1022, 224, 224, 3)


In [6]:
x_test_images=[]
#for i in range(100):
for i in range(x_test.shape[0]): 
    img = image.load_img('/home/snehao/dog_breed/train/'+x_test.iloc[i][0]+'.jpg',target_size=(224,224,3))
    arr=image.img_to_array(img)
    x_test_images.append(arr/255)
    
x_test_images=np.array(x_test_images)
print('y_shape',y_test.shape)
print('x_shape',x_test_images.shape)

y_shape (1023, 120)
x_shape (1023, 224, 224, 3)


In [7]:
files=!ls {"/home/snehao/dog_breed/Image_Classification/"}test
i=0
x_predict=[]
for i in range(len(files)):
    img1=image.load_img(f'{"/home/snehao/dog_breed/Image_Classification/"}test/{files[i]}',target_size=(224,224,3))
    arr=image.img_to_array(img1)
    x_predict.append(arr/255)
x_predict=np.array(x_predict)

In [8]:
from keras.layers import Concatenate,Add,BatchNormalization,Input,Dropout,Dense,Activation,Flatten,MaxPooling2D,Conv2D,ZeroPadding2D
from keras.models import Model,Sequential
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
base_model = InceptionV3(input_shape=(224,224,3),weights='imagenet', include_top=False,pooling='avg')
base_model1 = Xception(input_shape=(224,224,3),weights='imagenet', include_top=False,pooling='avg')
#base_model2 = VGG16(input_shape=(224,224,3),weights='imagenet', include_top=False,pooling='avg')
#base_model3 = VGG19(input_shape=(224,224,3),weights='imagenet', include_top=False,pooling='avg')
#base_model4 = ResNet50(input_shape=(224,224,3),weights='imagenet', include_top=False,pooling='avg')


In [9]:
ipred=base_model.predict(x)
xpred=base_model1.predict(x)
#v16pred=base_model2.predict(x)
#v19pred=base_model3.predict(x)
#rpred=base_model4.predict(x)

In [10]:
print(ipred.shape,xpred.shape)

(8177, 2048) (8177, 2048)


In [11]:
final=np.concatenate((ipred,xpred),axis=1)

In [12]:
print(final.shape)

(8177, 4096)


In [13]:
ifpred=base_model.predict(np.flip(x,axis=2))
xfpred=base_model1.predict(np.flip(x,axis=2))
#vf16pred=base_model2.predict(np.flip(x,axis=2))
#vf19pred=base_model3.predict(np.flip(x,axis=2))
#rfpred=base_model4.predict(np.flip(x,axis=2))

In [14]:
print(ifpred.shape,xfpred.shape)

(8177, 2048) (8177, 2048)


In [15]:
finalf=np.concatenate((ifpred,xfpred),axis=1)

In [16]:
ivpred=base_model.predict(x_val_images)
xvpred=base_model1.predict(x_val_images)
#vv16pred=base_model2.predict(x_val_images)
#vv19pred=base_model3.predict(x_val_images)
#rvpred=base_model4.predict(x_val_images)

In [17]:
print(ivpred.shape,xvpred.shape)

(1022, 2048) (1022, 2048)


In [18]:
finalv=np.concatenate((ivpred,xvpred),axis=1)

In [19]:
finalv.shape

(1022, 4096)

In [20]:
ivfpred=base_model.predict(np.flip(x_val_images,axis=2))
xvfpred=base_model1.predict(np.flip(x_val_images,axis=2))
#vvf16pred=base_model2.predict(np.flip(x_val_images,axis=2))
#vvf19pred=base_model1.predict(np.flip(x_val_images,axis=2))
#rvfpred=base_model2.predict(np.flip(x_val_images,axis=2))
finalvf=np.concatenate((ivfpred,xvfpred),axis=1)

In [21]:
import cv2

In [22]:
img=[]
for i in range(x.shape[0]):
    cropped_img = x[i][50:200, 50:200]
    img.append(cv2.resize(cropped_img,(224,224)))
img1=np.array(img)

In [23]:
img1.shape

(8177, 224, 224, 3)

In [24]:
imgv=[]
for i in range(x_val_images.shape[0]):
    cropped_img = x_val_images[i][50:200, 50:200]
    imgv.append(cv2.resize(cropped_img,(224,224)))
imgv1=np.array(imgv)

In [25]:
imgv1.shape

(1022, 224, 224, 3)

In [26]:
imgt=[]
for i in range(x_test_images.shape[0]):
    cropped_img = x_test_images[i][50:200, 50:200]
    imgt.append(cv2.resize(cropped_img,(224,224)))
imgt=np.array(imgt)

In [27]:
imgp=[]
for i in range(x_predict.shape[0]):
    cropped_img = x_predict[i][50:200, 50:200]
    imgp.append(cv2.resize(cropped_img,(224,224)))
imgp=np.array(imgp)

In [28]:
imgp.shape

(10357, 224, 224, 3)

In [29]:
itpred=base_model.predict(x_test_images)
xtpred=base_model1.predict(x_test_images)
#vt16pred=base_model2.predict(x_test_images)
#vt19pred=base_model3.predict(x_test_images)
#rtpred=base_model4.predict(x_test_images)
finalt=np.concatenate((itpred,xtpred),axis=1)
finalt.shape

(1023, 4096)

In [43]:
from keras import optimizers 
import keras.regularizers as regularizers 
input_shape=Input((4096,)) 

x1=Dense(120, activation='softmax')(input_shape) 
model=Model(inputs=input_shape,outputs=x1) 
print(model.summary()) 
#sgd=optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False) 
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 4096)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 120)               491640    
Total params: 491,640
Trainable params: 491,640
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
model.fit(final,y,epochs=100,validation_data=(finalv,y_val),batch_size=150)

Train on 8177 samples, validate on 1022 samples
Epoch 1/100
8177/8177 [==============================] - 0s - loss: 4.1342 - acc: 0.1817 - val_loss: 3.2750 - val_acc: 0.4217
Epoch 2/100
8177/8177 [==============================] - 0s - loss: 2.6340 - acc: 0.5948 - val_loss: 2.1467 - val_acc: 0.6712
Epoch 3/100
8177/8177 [==============================] - 0s - loss: 1.7504 - acc: 0.7351 - val_loss: 1.5410 - val_acc: 0.7407
Epoch 4/100
8177/8177 [==============================] - 0s - loss: 1.2942 - acc: 0.7793 - val_loss: 1.2305 - val_acc: 0.7671
Epoch 5/100
8177/8177 [==============================] - 0s - loss: 1.0533 - acc: 0.7997 - val_loss: 1.0612 - val_acc: 0.7847
Epoch 6/100
8177/8177 [==============================] - 0s - loss: 0.9121 - acc: 0.8136 - val_loss: 0.9560 - val_acc: 0.7896
Epoch 7/100
8177/8177 [==============================] - 0s - loss: 0.8188 - acc: 0.8222 - val_loss: 0.8891 - val_acc: 0.7935
Epoch 8/100
8177/8177 [==============================] - 0s - loss: 0.

8177/8177 [==============================] - 0s - loss: 0.2396 - acc: 0.9488 - val_loss: 0.6217 - val_acc: 0.8072
Epoch 66/100
8177/8177 [==============================] - 0s - loss: 0.2372 - acc: 0.9490 - val_loss: 0.6204 - val_acc: 0.8092
Epoch 67/100
8177/8177 [==============================] - 0s - loss: 0.2344 - acc: 0.9499 - val_loss: 0.6207 - val_acc: 0.8043
Epoch 68/100
8177/8177 [==============================] - 0s - loss: 0.2318 - acc: 0.9506 - val_loss: 0.6215 - val_acc: 0.8072
Epoch 69/100
8177/8177 [==============================] - 0s - loss: 0.2292 - acc: 0.9521 - val_loss: 0.6223 - val_acc: 0.8053
Epoch 70/100
8177/8177 [==============================] - 0s - loss: 0.2266 - acc: 0.9535 - val_loss: 0.6226 - val_acc: 0.8072
Epoch 71/100
8177/8177 [==============================] - 0s - loss: 0.2242 - acc: 0.9544 - val_loss: 0.6227 - val_acc: 0.8063
Epoch 72/100
8177/8177 [==============================] - 0s - loss: 0.2218 - acc: 0.9551 - val_loss: 0.6221 - val_acc: 0.80

In [ ]:
result=model.evaluate(x=finalt,y=y_test)

In [ ]:
result

In [ ]:
model.save_weights('original1.h5')


In [34]:
ippred=base_model.predict(x_predict)
xppred=base_model1.predict(x_predict)
#vp16pred=base_model2.predict(x_predict)
#vp19pred=base_model3.predict(x_predict)
#rppred=base_model4.predict(x_predict)
finalp=np.concatenate((ippred,xppred),axis=1)
finalp.shape

(10357, 4096)

In [35]:
predo=model.predict(finalp)

In [37]:
model.fit(finalf,y,validation_data=(finalvf,y_val),epochs=100,batch_size=100)

Train on 8177 samples, validate on 1022 samples
Epoch 1/100
8177/8177 [==============================] - 0s - loss: 3.8069 - acc: 0.2671 - val_loss: 2.6903 - val_acc: 0.5636
Epoch 2/100
8177/8177 [==============================] - 0s - loss: 1.9975 - acc: 0.6907 - val_loss: 1.5676 - val_acc: 0.7211
Epoch 3/100
8177/8177 [==============================] - 0s - loss: 1.2555 - acc: 0.7738 - val_loss: 1.1524 - val_acc: 0.7573
Epoch 4/100
8177/8177 [==============================] - 0s - loss: 0.9614 - acc: 0.8045 - val_loss: 0.9759 - val_acc: 0.7769
Epoch 5/100
8177/8177 [==============================] - 0s - loss: 0.8156 - acc: 0.8184 - val_loss: 0.8876 - val_acc: 0.7838
Epoch 6/100
8177/8177 [==============================] - 0s - loss: 0.7296 - acc: 0.8288 - val_loss: 0.8260 - val_acc: 0.7916
Epoch 7/100
8177/8177 [==============================] - 0s - loss: 0.6693 - acc: 0.8387 - val_loss: 0.7872 - val_acc: 0.7906
Epoch 8/100
8177/8177 [==============================] - 0s - loss: 0.

8177/8177 [==============================] - 0s - loss: 0.1755 - acc: 0.9687 - val_loss: 0.6631 - val_acc: 0.7955
Epoch 66/100
8177/8177 [==============================] - 0s - loss: 0.1730 - acc: 0.9691 - val_loss: 0.6636 - val_acc: 0.7955
Epoch 67/100
8177/8177 [==============================] - 0s - loss: 0.1709 - acc: 0.9710 - val_loss: 0.6627 - val_acc: 0.7926
Epoch 68/100
8177/8177 [==============================] - 0s - loss: 0.1687 - acc: 0.9711 - val_loss: 0.6624 - val_acc: 0.7935
Epoch 69/100
8177/8177 [==============================] - 0s - loss: 0.1664 - acc: 0.9715 - val_loss: 0.6656 - val_acc: 0.7916
Epoch 70/100
8177/8177 [==============================] - 0s - loss: 0.1643 - acc: 0.9731 - val_loss: 0.6642 - val_acc: 0.7916
Epoch 71/100
8177/8177 [==============================] - 0s - loss: 0.1622 - acc: 0.9724 - val_loss: 0.6647 - val_acc: 0.7935
Epoch 72/100
8177/8177 [==============================] - 0s - loss: 0.1602 - acc: 0.9740 - val_loss: 0.6652 - val_acc: 0.79

In [ ]:
model.save_weights('flip1.h5')

In [38]:
ifppred=base_model.predict(np.flip(x_predict,axis=2))
xfppred=base_model1.predict(np.flip(x_predict,axis=2))
#vfp16pred=base_model2.predict(np.flip(x_predict,axis=2))
#vfp19pred=base_model3.predict(np.flip(x_predict,axis=2))
#rfppred=base_model4.predict(np.flip(x_predict,axis=2))

In [39]:
predf=model.predict(np.concatenate((ifppred,xfppred),axis=1))

In [40]:
icpred=base_model.predict(img1)
xcpred=base_model1.predict(img1)
#vc16pred=base_model2.predict(img1)
#vc19pred=base_model3.predict(img1)
#rcpred=base_model4.predict(img1)
finalc=np.concatenate((icpred,xcpred),axis=1)
finalc.shape

(8177, 4096)

In [41]:
icvpred=base_model.predict(imgv1)
xcvpred=base_model1.predict(imgv1)
#vcv16pred=base_model2.predict(imgv1)
#vcv19pred=base_model3.predict(imgv1)
#rcvpred=base_model4.predict(imgv1)
finalcv=np.concatenate((icvpred,xcvpred),axis=1)
finalcv.shape

(1022, 4096)

In [42]:
icppred=base_model.predict(imgp)
xcppred=base_model1.predict(imgp)
#vcp16pred=base_model2.predict(imgp)
#vcp19pred=base_model1.predict(imgp)
#rcppred=base_model2.predict(imgp)
finalct=np.concatenate((icppred,xcppred),axis=1)
finalct.shape

(10357, 4096)

In [44]:
model.fit(finalc,y,validation_data=(finalcv,y_val),epochs=100,batch_size=100)

Train on 8177 samples, validate on 1022 samples
Epoch 1/100
8177/8177 [==============================] - 0s - loss: 3.8885 - acc: 0.2492 - val_loss: 2.8667 - val_acc: 0.5450
Epoch 2/100
8177/8177 [==============================] - 0s - loss: 2.1666 - acc: 0.6748 - val_loss: 1.7230 - val_acc: 0.7035
Epoch 3/100
8177/8177 [==============================] - 0s - loss: 1.3731 - acc: 0.7700 - val_loss: 1.2453 - val_acc: 0.7476
Epoch 4/100
8177/8177 [==============================] - 0s - loss: 1.0350 - acc: 0.8013 - val_loss: 1.0263 - val_acc: 0.7691
Epoch 5/100
8177/8177 [==============================] - 0s - loss: 0.8658 - acc: 0.8196 - val_loss: 0.9104 - val_acc: 0.7740
Epoch 6/100
8177/8177 [==============================] - 0s - loss: 0.7648 - acc: 0.8342 - val_loss: 0.8385 - val_acc: 0.7808
Epoch 7/100
8177/8177 [==============================] - 0s - loss: 0.6964 - acc: 0.8444 - val_loss: 0.7963 - val_acc: 0.7896
Epoch 8/100
8177/8177 [==============================] - 0s - loss: 0.

8177/8177 [==============================] - 0s - loss: 0.1870 - acc: 0.9676 - val_loss: 0.5859 - val_acc: 0.8170
Epoch 66/100
8177/8177 [==============================] - 0s - loss: 0.1845 - acc: 0.9692 - val_loss: 0.5867 - val_acc: 0.8190
Epoch 67/100
8177/8177 [==============================] - 0s - loss: 0.1824 - acc: 0.9700 - val_loss: 0.5858 - val_acc: 0.8180
Epoch 68/100
8177/8177 [==============================] - 0s - loss: 0.1801 - acc: 0.9709 - val_loss: 0.5857 - val_acc: 0.8190
Epoch 69/100
8177/8177 [==============================] - 0s - loss: 0.1777 - acc: 0.9710 - val_loss: 0.5858 - val_acc: 0.8190
Epoch 70/100
8177/8177 [==============================] - 0s - loss: 0.1755 - acc: 0.9710 - val_loss: 0.5855 - val_acc: 0.8229
Epoch 71/100
8177/8177 [==============================] - 0s - loss: 0.1734 - acc: 0.9725 - val_loss: 0.5858 - val_acc: 0.8209
Epoch 72/100
8177/8177 [==============================] - 0s - loss: 0.1712 - acc: 0.9726 - val_loss: 0.5859 - val_acc: 0.81

In [ ]:
model.save_weights('zoom1.h5')

In [45]:
predc=model.predict(finalct)

In [46]:
final_predic=(predo+predf+predc)/3

In [ ]:
np.savetxt('3_models.csv',final_predic,fmt='%f',delimiter=',')